In [20]:
"""Esta funcion esta diseñada para calcular el costo de los compresores de diafragma.
Se basa en las graficas del libro "Analysis, Synthesis, and Design of Chemical Processes
de Richard Turton 

Para que el codigo funcione se necesita ingresar una tabla con los siguientes datos:
- El número de compresores
- La potencia de cada compresor (kW)"""

'Esta funcion esta diseñada para calcular el costo de los compresores de diafragma.\nSe basa en las graficas del libro "Analysis, Synthesis, and Design of Chemical Processes\nde Richard Turton \n\nPara que el codigo funcione se necesita ingresar una tabla con los siguientes datos:\n- El número de compresores\n- La potencia de cada compresor (kW)'

In [ ]:
#Este script es para la prueba del objeto programdo que debe calcular el precio de los compresores 

import pandas as pd

data = {
    "Compresor": ["No.1", "No.2", "No.3","Total"],
    "T(K)": [300, 350, 400, 289],
    "P(bar)": [78.14, 125.25, 172.3,173.3],
    "Q(kW)": [50, 60, 70,50],
    "W (kW)": [605.3, 327.322, 206.3,1173.8],
    "Q+W (kW)": [150, 170, 190,1200]
}

prueba = pd.DataFrame(data)
prueba

,Compresor,T(K),P(bar),Q(kW),W (kW),Q+W (kW)
0,No.1,300,78.14,50,605.300,150
1,No.2,350,125.25,60,327.322,170
2,No.3,400,172.30,70,206.300,190
3,Total,289,173.30,50,1173.800,1200


In [ ]:
class PriceCalculator:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def calculate_price(self):
        prices = []
        cepci_2001=397
        cepci_2024=798.8
        for index, row in self.dataframe.iterrows():
            if index < len(self.dataframe) - 1:  # For all rows except the last one
                w_value = row["W (kW)"]
                price = (7E-12 * w_value**4 - 6E-08 * w_value**3 + 0.0002 * w_value**2 - 0.2777 * w_value + 426)*(cepci_2024/cepci_2001)
                prices.append(price)
            else:  # For the last row
                total_price = sum(prices)
                prices.append(total_price)
        self.dataframe["Price"] = prices
        return self.dataframe

#Try to use the class with prueba dataframe
calculator = PriceCalculator(prueba)
updated_prueba = calculator.calculate_price()
updated_prueba

,Compresor,T(K),P(bar),Q(kW),W (kW),Q+W (kW),Price
0,No.1,300,78.14,50,605.300,150,641.492753
1,No.2,350,125.25,60,327.322,170,713.299870
2,No.3,400,172.30,70,206.300,190,757.971266
3,Total,289,173.30,50,1173.800,1200,2112.763889
